In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pwd

'/content'

In [ ]:
%cd drive/MyDrive/Github/ml_dl_tweets_sentiment/_10_51_running_mlflow_on_colab

/content/drive/MyDrive/Github/ml_dl_tweets_sentiment/_10_51_running_mlflow_on_colab


In [ ]:
!pip install mlflow

     |████████████████████████████████| 16.5 MB 6.7 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 146 kB 58.5 MB/s 
     |████████████████████████████████| 596 kB 65.5 MB/s 
     |████████████████████████████████| 181 kB 48.5 MB/s 
     |████████████████████████████████| 210 kB 53.4 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.16.4-py3-none-any.whl size=106877 sha256=ae34a1daf80f4a05d9b538ba8bc5b337f0c4a012185d5c7ef9d8f412e1cb7cdf
  Stored in directory: /root/.cache/pip/wheels/a2/a1/6d/fa1d22ea25ed8593887437fe1c7e00f6ef307fc240ccd4dc5c
Successfully built databricks-cli
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfu

In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 8.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=d2491b20534f05b4e1333ca4c22d0ce085f68f5d6ba9278429880840a592696a
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [ ]:
from pyngrok import ngrok


In [ ]:
!ngrok authtoken token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!cat /root/.ngrok2/ngrok.yml

In [ ]:
ngrok.kill()


In [ ]:
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)


In [ ]:
MLflow_UI_url = ngrok_tunnel.public_url

In [ ]:
print("MLflow UI ", MLflow_UI_url)


MLflow UI  https://b818-34-125-190-255.ngrok.io


In [ ]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn

if __name__ == "__main__":

    mlflow.set_experiment(experiment_name="mlflow demo")
    
    training_data = pd.read_csv('../_09_47_local_model_store/storepurchasedata.csv')
    print("loaded training data")

    training_data.describe()
    mlflow.log_param("training percentage",60)

    mlflow.log_param("dataset shape",training_data.shape)
    
    X = training_data.iloc[:, :-1].values
    y = training_data.iloc[:,-1].values
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.4,random_state=0)
    
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    print("Completed Feature Scaling")
    
    from sklearn.neighbors import KNeighborsClassifier
    # minkowski is for ecledian distance
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    
    # Model training
    classifier.fit(X_train, y_train)
    print("Model trained")
   
    y_pred = classifier.predict(X_test)
    y_prob = classifier.predict_proba(X_test)[:,1]
    
    from sklearn.metrics import confusion_matrix
    
    cm = confusion_matrix(y_test, y_pred)
    
    from sklearn.metrics import accuracy_score
    
    model_accuracy = accuracy_score(y_test,y_pred)
    
    print(model_accuracy)
    
    mlflow.log_metric("accuracy", model_accuracy)
    mlflow.sklearn.log_model(classifier, "model")
    
    

2022/03/25 18:40:06 INFO mlflow.tracking.fluent: Experiment with name 'mlflow demo' does not exist. Creating a new experiment.


loaded training data
Completed Feature Scaling
Model trained
0.8125


In [ ]:
#ngrok.disconnect(MLflow_UI_url)